In [14]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torchvision import datasets, transforms
# import numpy as np
# Import main packages
import os
import copy
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from typing import Optional, Callable
from sklearn.metrics import accuracy_score, f1_score
from sklearn.covariance import LedoitWolf
import json

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [44]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(256*256*3, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 16)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))

In [71]:
class CoinDataset(Dataset):
    """DHMC dataset using 2 classes"""

    def __init__(self, features_path : str, label_path,train : bool = False) -> None:
        """
        Attributes:
            raw_data (list of dict): (M) List of M slides raw data as dictionaries. 
            train (bool): True if data are the training set. False otherwise
            
        Args:
            features_path (str): The path to the features file
            train (bool): Whether it is the training dataset or not
        """
        
        super().__init__()
        # Load raw data from path
        
        # self.raw_data = torch.load(features_path)
        # load labels from json file
        self.raw_data = []
        data_json = None
        with open(label_path, 'r') as f:
            data_json = json.load(f)
        for x in data_json:
            filename = x["filename"]
            img_path = f"{features_path}/{filename}"
            img = Image.open(img_path)
            img = img.resize((256, 256))
            img = np.array(img)
            img = img.flatten()
            self.raw_data.append({"image_features": img, "label": int(x["value"])})

            #open image
        # Set if training or not
        self.train = train

    def __len__(self) -> int:
        """Returns the length of the dataset

        Returns:
            int: The length M of the dataset
        """

        n_data = 0
        
        # ------------------
        # Your code here ... 
        # ------------------
        n_data = len(self.raw_data)
        return n_data
    
    def __getitem__(self, index : int):
        """Returns the entry at index from the dataset

        Args:
            index (int): the requested entry index of the dataset

        Returns:
            features (torch.Tensor): (N, d) Feature tensor of the selected slide with N patches and d feature dimensions
            label (int): Ground truth label {0, ..., n_classes}
            wsi_id (str): Name of the WSI as "DHMC_xxx" where xxx is a unique id of the slide (train == False only)
            coordinates (torch.Tensor): (N, 2) xy coordinates of the N patches of the selected slide (train == False only)
        """

        features = None
        label = None
        wsi_id = None
        coordinates = None
        
        # ------------------
        # Your code here ... 
        # ------------------
        features = torch.tensor(self.raw_data[index]["image_features"]).float()
        label = torch.tensor(self.raw_data[index]["label"])
        # coordinates = self.raw_data[index]["patch_coordinates"]
        # wsi_id = self.raw_data[index]["wsi_id"]
        # if self.train:
        return features, label
        # else:
        #     return features, label, wsi_id, coordinates

In [72]:
# Load all the img from output folder and make a dataset
coinDataset = CoinDataset('./output-1/', "data-1.json", train=True)
# trans = tranosforms.Compose([transforms.Resize(256),transforms.ToTensor()])
train_loader = DataLoader(coinDataset, batch_size=1, shuffle=True, num_workers=1)
# dataset = datasets.ImaVgeFolder('./output-1/', transform=transforms.ToTensor())

In [83]:
from torchvision.models import resnet50, ResNet50_Weights

model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
def train(model, train_loader, optimizer, criterion, epochs=10):
    for epoch in range(epochs):
        for i, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            # data = data.view(-1, 256*256)
            optimizer.zero_grad()
            output = model(data)
            # output.softmax(0)
            # output = torch.round(output)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, i * len(data), len(train_loader.dataset),
                    100. * i / len(train_loader), loss.item()))
train(model, train_loader, torch.optim.Adam(model.parameters(), lr=0.04), nn.BCELoss(), epochs=10)

NameError: name 'ResNet50_Weights' is not defined